In [2]:
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame 

In [3]:
url = "https://www.coupang.com/np/search?q=%EB%85%B8%ED%8A%B8%EB%B6%81&listSize=72&channel=user"
userAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
session = requests.Session()
session.headers.update({
    "Referer": "",
    "User-Agent": userAgent,
    "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"
})
r = session.get(url)
if r.status_code !=200:
    msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
    raise Exception(msg)
r.encoding = "utf-8"
soup = BeautifulSoup(r.text)

In [5]:
product = soup.select(".search-product-wrap")
print("검색결과 수 :", len(product))

검색결과 수 : 72


In [7]:
data = []
for p in product:
    nameEL = p.select("name")
    if nameEL:
        name = nameEL[0].text.strip()
    else:
        name = None
    basePriceEl = p.select(".base-price")
    if basePriceEl:
        basePrice = int(basePriceEl[0].text.strip().replace(",",""))
    else:
        basePrice = None
    discountEl = p.select(".instant-discount-rate")
    if discountEl:
        discount = int(discountEl[0].text.strip().replace("%",""))/100
    else:
        discount = None
    priceEl = p.select(".price-value")
    if priceEl:
        price = int(priceEl[0].text.strip().replace(",","")) 
    else:
        price = None
    ratingEl = p.select(".rating")
    if ratingEl:
        rating = float(ratingEl[0].text.strip()) 
    else:
        basePrice = None
    reviewEl = p.select(".rating-total-count")
    if reviewEl:
        review = reviewEl[0].text.strip()
        review = int(review[1:-1])
    else:
        basePrice = None
    imageEl = p.select(".search-product-wrap-img")
    if imageEl:
        if "data-img-src" in imageEl[0].attrs: 
            image = "http:" + imageEl[0].attrs['data-img-src']
        else:
            image = "http:" + imageEl[0].attrs['src']
    else:
        image = None
    item = {
        "상품명": name,
        "가격" : basePrice,
        "할인율": discount,
        "할인가": price,
        "별점": rating,
        "리뷰수": review,
        "상품이미지": image
    }
    data.append(item)
df=DataFrame(data)